In [2]:
import numpy as np
import pandas as pd
import json
from datetime import datetime, timedelta
from dateutil.parser import parse

<h2>Functions</h2>

In [617]:
# Get the Names of Employees in the HR Dataset
# param df (dataframe) - Dataframe with names
# param column (str) - Column can be 'Employee Name' or 'Manager Name'
# Returns names (list) - Array of names (['First', 'Last'])
def df_get_names(df, column='Employee Name'):
    names = [df[[column]].iloc[i][0] for i in range(df.shape[0])]
    for i in range(len(names)):
        try:
            full_name = names[i].split(',')
            first = full_name[1].strip().split()[0]
            last = full_name[0]
            names[i] = [first, last]
        except:
            names[i] = ['Jeremy', 'Prater']
    return names

# Split the Name Column in the Dataframe
# param df (dataframe) - Dataframe with names
# Returns updated_df (dataframe) - Dataframe with a 'first_name' and 'last_name' column
def df_split_name_col(df):
    names = df_get_names(df, 'emp_name')
    for i in range(len(names)):
        df.at[i, 'first_name'] = names[i][0]
        df.at[i, 'last_name'] = names[i][1]
    return df

# Reformat Date from 'MM/DD/YY' to 'YYYY-MM-DD' for DOB, DOH, DOT
# param date (str) - Date to reformat in the form 'MM/DD/YYYY'
# param df (dataframe) - Dataframe with Dates
def reformat_date(date_str):
    date_split = date_str.split('/')
    if(int(date_split[2]) < 20): date_split[2] = '20' + date_split[2]
    else: date_split[2] = '19' + date_split[2]
    date_str = '/'.join(date_split)
    return datetime.strptime(date_str, '%m/%d/%Y').strftime('%Y-%m-%d')

# Reformat Date from 'MM/DD/YYYY' to 'YYYY-MM-DD' for DOB, DOH, DOT
# param df (dataframe) - Dataframe with Dates
# Returns updated_df (dataframe) - Dataframe with a 'first_name' and 'last_name' column
def df_reformat_date(df):
    cols = ['doh', 'dob', 'dot']
    for col in cols:
        for i in range(len(df)):
            if(df[col][i] != '1800-01-01' ): df.at[i, col] = reformat_date(df[col][i])
    return df

<h3>File Writing</h3>

In [421]:
# Convert a Knowledge Base Dataframe to JSON File
# param df (dataframe) -  Dataframe to write to file
def df_to_json_file(df):
    json_dict = df.to_dict(orient='records')
    with open("../hr_assistant/data/user_data.json", "w+") as f:
        json_str = json.dumps(json_dict, indent=4)
        f.write(json_str)

<h3>Knowledge Base Query Helpers</h3>

In [656]:
# Get a List of Names from a QA Result
# param qa_out (list) Output of QA from a query
def _get_names(qa_out):
    return [out['emp_name'] for out in qa_out]
  
# Filter the output of the Question Answerer
# param ent (str) Entity to filter on
# param val (str) Value to filter for
# param qa_out (list) List of Json Objects Representing Users
# Return qa_out_filtered (list) List if JSON Objects filtered by entity and value
def _categ_filter(ent, val, qa_out): 
    return [x for x in users if x[ent] == val]

# Convert Date object to Str Date Format 'YYYY-MM-DD'
# param date_obj (Datetime object) - Datetime object to convert
def _d_to_ymd(d): return d.strftime('%Y-%m-%d')

# Get the Datetime Str for a Certain number of Years, Months, Days ago from the present time
# param years (int) - Number of years ago 
# param months (int) - Number of months ago 
# param weeks (int) - Number of weeks ago
# param days (int) - Number of days ago 
# return date_obj (Datetime Object) - Datetime Object
def _get_ago_t(years=0, months=0, weeks=0, days=0):
    total_days = years*52*7 + months*4*7 + weeks*7 + days
    d = get_now_time() - timedelta(days=total_days)
    return d

# Get the Datetime Object for the current time
# return date_obj (Datetime Object) - Datetime Object
def _get_now_t(): return datetime.now()

# Get the Datetime Str for the current time
# return date_str (str) - Str of date in the format'YYYY-MM-DD'
def _get_now_t_str(): return d_to_ymd(get_now_time())


# Filter the output of the Question Answerer by Date
# param date_type (str) Date Type to Filter On: 'doh', 'dob', 'dot'
# param qa_out (list) List of Json Objects Representing Users
# param start_d (str) Start Date in the format 'YYYY-MM-DD'
# param end_d (str) End Date in the format 'YYYY-MM-DD'
# Return qa_out_filtered (list) List if JSON Objects filtered by Date Type
def _filter_by_d(date_type, qa_out, start_d='1900-01-01', end_d=get_now_t()):
    s = qa.build_search(index='user_data_3')
    return s.filter(field=date_type, lte=end_d, gte=start_d).execute()

# Filter the output of the Question Answerer by Date
# param d_type (str) Date Type to Filter On: 'doh', 'dob', 'dot'
# param qa_out (list) List of Json Objects Representing Users
# param gt (str) Greater than Start Date in the format 'YYYY-MM-DD'
# param gte (str) Greater than or equal to Start Date in the format 'YYYY-MM-DD'
# param lt (str) Greater than Start Date in the format 'YYYY-MM-DD'
# param lte (str) Greater than or equal to Start Date in the format 'YYYY-MM-DD'
# Return qa_out_filtered (list) List if JSON Objects filtered by Date Type
def _filter_by_d_custom(d_type, qa_out, gt, gte, lt, lte):
    # Less than (or equal to)
    if lt is not None:
        lt = _ymd_to_d(lt)
        qa_out = [x for x in qa_out if _ymd_to_d(x[d_type]) < lt]
    elif lte is not None:
        lte = _ymd_to_d(lte)
        qa_out = [x for x in qa_out if _ymd_to_d(x[d_type]) <= lte]
    # Greater than (or equal to)
    if gt is not None:
        gt = _ymd_to_d(gt)
        qa_out = [x for x in qa_out if _ymd_to_d(x[d_type]) > gt]
    elif gte is not None:
        gte = _ymd_to_d(gte)
        qa_out = [x for x in qa_out if _ymd_to_d(x[d_type]) >= gte]
    # Remove "Null" values in DOT that were replaced with "1800-01-01"
    if d_type == 'dot': qa_out = [x for x in qa_out if _ymd_to_d(x[d_type]) > _ymd_to_d("1800-01-01")] 
    return qa_out

# Function Helper that does Sum, Average and Percent Calculations
# param qa_out (list) List of Json Objects Representing Users
# param func (str) - Function Type: 'avg','sum', 'ct', 'pct'
# param num_col (str) - Numerical Column Type : 'money', or 'age'
# returns result (float) - Resulting Value from function operation
def _agg_function(qa_out, func='avg', num_col='money'):
    if(func=='avg'): return np.mean([emp[num_col] for emp in qa_out])
    elif(func=='sum'): return np.sum([emp[num_col] for emp in qa_out])
    elif(func=='ct'): return len(qa_out)
    elif(func=='pct'): return len(qa_out)/300

# Get the Salary Amount Based on a Recurring Period of Time
# param recur_ent (str): 'yearly', 'monthly', 'weely', 'daily', 'hourly'
# param money (float): Hourly Salary of an employee
def _get_interval_amount(recur_ent, money):
    intv_mult = { "yearly": 12*4*5*8, "monthly": 4*5*8, "weekly":5*8, "daily": 8,"hourly": 1}
    return round(intv_mult[recur_ent] * money, 2)         

<h2>Workflow</h2>

<h4>Create Json</h4>

In [618]:
kb = pd.read_csv('core_dataset.csv')
kb.fillna("1800-01-01", inplace=True)
kb = df_split_name_col(kb)
kb = df_reformat_date(kb)
kb

,emp_name,id,state,dob,age,sex,maritaldesc,citizendesc,racedesc,doh,...,rft,employment_status,department,position,money,manager,employee_source,performance_score,first_name,last_name
0,"Brown, Mia",1103024456,MA,1985-11-24,32,Female,Married,US Citizen,Black or African American,2008-10-27,...,N/A - still employed,Active,Admin Offices,Accountant I,28.50,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets,Mia,Brown
1,"LaRotonda, William",1106026572,MA,1984-04-26,33,Male,Divorced,US Citizen,Black or African American,2014-01-06,...,N/A - still employed,Active,Admin Offices,Accountant I,23.00,Brandon R. LeBlanc,Website Banner Ads,Fully Meets,William,LaRotonda
2,"Steans, Tyrone",1302053333,MA,1986-09-01,31,Male,Single,US Citizen,White,2014-09-29,...,N/A - still employed,Active,Admin Offices,Accountant I,29.00,Brandon R. LeBlanc,Internet Search,Fully Meets,Tyrone,Steans
3,"Howard, Estelle",1211050782,MA,1985-09-16,32,Female,Married,US Citizen,White,2015-02-16,...,N/A - still employed,Active,Admin Offices,Administrative Assistant,21.50,Brandon R. LeBlanc,Pay Per Click - Google,N/A- too early to review,Estelle,Howard
4,"Singh, Nan",1307059817,MA,1988-05-19,29,Female,Single,US Citizen,White,2015-05-01,...,N/A - still employed,Active,Admin Offices,Administrative Assistant,16.56,Brandon R. LeBlanc,Website Banner Ads,N/A- too early to review,Nan,Singh
5,"Smith, Leigh Ann",711007713,MA,1987-06-14,30,Female,Married,US Citizen,Asian,2011-09-26,...,career change,Voluntarily Terminated,Admin Offices,Administrative Assistant,20.50,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets,Leigh,Smith
6,"LeBlanc, Brandon R",1102024115,MA,1984-06-10,33,Male,Married,US Citizen,White,2016-01-05,...,N/A - still employed,Active,Admin Offices,Shared Services Manager,55.00,Janet King,Monster.com,Fully Meets,Brandon,LeBlanc
7,"Quinn, Sean",1206043417,MA,1984-11-06,33,Male,Married,Eligible NonCitizen,Black or African American,2011-02-21,...,career change,Voluntarily Terminated,Admin Offices,Shared Services Manager,55.00,Janet King,Diversity Job Fair,Fully Meets,Sean,Quinn
8,"Boutwell, Bonalyn",1307060188,MA,1987-04-04,30,Female,Married,US Citizen,Asian,2015-02-16,...,N/A - still employed,Active,Admin Offices,Sr. Accountant,34.95,Brandon R. LeBlanc,Diversity Job Fair,90-day meets,Bonalyn,Boutwell
9,"Foster-Baker, Amy",1201031308,MA,1979-04-16,38,Female,Married,US Citizen,White,2009-01-05,...,N/A - still employed,Active,Admin Offices,Sr. Accountant,34.95,Board of Directors,Other,Fully Meets,Amy,Foster-Baker


In [634]:
df_to_json_file(kb)

In [540]:
#kb['doh'][3]

<h4>Knowledge Base</h4>

In [4]:
from mindmeld.components import QuestionAnswerer
qa = QuestionAnswerer(app_path='hr_assistant')
qa.load_kb('hr_assistant', 'user_data_3', '../hr_assistant/data/user_data.json')

100%|██████████| 301/301 [00:00<00:00, 333.34it/s]


In [5]:
qa_out = qa.get(index='user_data_3', sex='Male')
qa_out

[{'rft': 'no-call, no-show',
  'performance_score': 'N/A- too early to review',
  'citizendesc': 'US Citizen',
  'manager': 'Simon Roup',
  'sex': 'Male',
  'maritaldesc': 'Married',
  'emp_name': 'Hernandez, Daniff',
  'dot': '2015-02-22',
  'last_name': 'Hernandez',
  'racedesc': 'Black or African American',
  'money': 40.1,
  'dob': '1986-08-07',
  'employee_source': 'Employee Referral',
  'id': 1410071156,
  'state': 'MA',
  'employment_status': 'Terminated for Cause',
  'position': 'Database Administrator',
  'doh': '2015-02-16',
  'department': 'IT/IS',
  'first_name': 'Daniff',
  'age': 31},
 {'rft': 'N/A - still employed',
  'performance_score': 'N/A- too early to review',
  'citizendesc': 'US Citizen',
  'manager': 'Simon Roup',
  'sex': 'Male',
  'maritaldesc': 'Married',
  'emp_name': 'Rogers, Ivan',
  'dot': '1800-01-01',
  'last_name': 'Rogers',
  'racedesc': 'White',
  'money': 42.2,
  'dob': '1986-08-26',
  'employee_source': 'Pay Per Click - Google',
  'id': 1203032255,

In [6]:
#qa._kb_field_info.get(field='dob')

In [11]:
s = qa.build_search(index='user_data_3')

In [13]:
s.filter(field='doh', lte='2019-01-01').execute()

[{'rft': 'N/A - still employed',
  'performance_score': 'N/A- too early to review',
  'citizendesc': 'US Citizen',
  'manager': 'Brandon R. LeBlanc',
  'sex': 'Female',
  'maritaldesc': 'Single',
  'emp_name': 'Singh, Nan ',
  'dot': '1800-01-01',
  'last_name': 'Singh',
  'racedesc': 'White',
  'money': 16.56,
  'dob': '1988-05-19',
  'employee_source': 'Website Banner Ads',
  'id': 1307059817,
  'state': 'MA',
  'employment_status': 'Active',
  'position': 'Administrative Assistant',
  'doh': '2015-05-01',
  'department': 'Admin Offices',
  'first_name': 'Nan',
  'age': 29},
 {'rft': 'N/A - still employed',
  'performance_score': '90-day meets',
  'citizendesc': 'US Citizen',
  'manager': 'Simon Roup',
  'sex': 'Male',
  'maritaldesc': 'Married',
  'emp_name': 'Simard, Kramer',
  'dot': '1800-01-01',
  'last_name': 'Simard',
  'racedesc': 'White',
  'money': 30.2,
  'dob': '1970-02-08',
  'employee_source': 'Employee Referral',
  'id': 808010278,
  'state': 'MA',
  'employment_status

In [8]:
qa_out = s.query(sex='Male').execute(size=300)

In [9]:
qa_out

[{'rft': 'no-call, no-show',
  'performance_score': 'N/A- too early to review',
  'citizendesc': 'US Citizen',
  'manager': 'Simon Roup',
  'sex': 'Male',
  'maritaldesc': 'Married',
  'emp_name': 'Hernandez, Daniff',
  'dot': '2015-02-22',
  'last_name': 'Hernandez',
  'racedesc': 'Black or African American',
  'money': 40.1,
  'dob': '1986-08-07',
  'employee_source': 'Employee Referral',
  'id': 1410071156,
  'state': 'MA',
  'employment_status': 'Terminated for Cause',
  'position': 'Database Administrator',
  'doh': '2015-02-16',
  'department': 'IT/IS',
  'first_name': 'Daniff',
  'age': 31},
 {'rft': 'N/A - still employed',
  'performance_score': 'N/A- too early to review',
  'citizendesc': 'US Citizen',
  'manager': 'Simon Roup',
  'sex': 'Male',
  'maritaldesc': 'Married',
  'emp_name': 'Rogers, Ivan',
  'dot': '1800-01-01',
  'last_name': 'Rogers',
  'racedesc': 'White',
  'money': 42.2,
  'dob': '1986-08-26',
  'employee_source': 'Pay Per Click - Google',
  'id': 1203032255,

In [10]:
qa_out[0]['doh']

'2015-02-16'

In [572]:
war_start = '2011-01-03'
datetime.strptime(war_start, '%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

In [576]:
_ymd_to_d(qa_out[0]['doh'])

datetime.datetime(2015, 2, 16, 0, 0)

In [657]:
# for user in qa_out:
#     print(datetime.strptime((user['doh'])))

In [658]:
#[x for x in qa_out if _ymd_to_d(x['dob']) > _ymd_to_d('1990-01-01')]

In [549]:
date_str = qa_out[0]['dob']

In [557]:
_ymd_to_d(date_str) == _ymd_to_d(date_str)

True

In [608]:
datetime.strptime("1800-01-01", '%Y-%m-%d')

datetime.datetime(1800, 1, 1, 0, 0)

In [654]:
_get_interval_amount("daily", 32.4545)

259.64

In [1]:
_filter_by_d_custom(d_type='dot', qa_out=qa_out, gt="1980-01-01", gte=None, lt=None, lte="2020-01-01")

NameError: name '_filter_by_d_custom' is not defined

In [494]:
len(s.query(sex="Female", size=300).execute() )  #.filter(field="age", lte='100', size=300).filter(field="money", gte=10, size=300).execute())

10

In [497]:
users = qa.get(index='user_data_3', sex='Female',size=301)

In [498]:
qa.get(index='user_data_3', sex='Female',size=301)

[{'rft': 'N/A - still employed',
  'performance_score': 'Fully Meets',
  'citizendesc': 'US Citizen',
  'manager': 'Brandon R. LeBlanc',
  'sex': 'Female',
  'maritaldesc': 'Married',
  'emp_name': 'Brown, Mia',
  'dot': '-1',
  'last_name': 'Brown',
  'racedesc': 'Black or African American',
  'money': 28.5,
  'dob': '1985-11-24',
  'employee_source': 'Diversity Job Fair',
  'id': 1103024456,
  'state': 'MA',
  'employment_status': 'Active',
  'position': 'Accountant I',
  'doh': '2008-10-27',
  'department': 'Admin Offices',
  'first_name': 'Mia',
  'age': 32},
 {'rft': 'no-call, no-show',
  'performance_score': 'Fully Meets',
  'citizendesc': 'US Citizen',
  'manager': 'Simon Roup',
  'sex': 'Female',
  'maritaldesc': 'Single',
  'emp_name': 'Goble, Taisha',
  'dot': '2015-03-15',
  'last_name': 'Goble',
  'racedesc': 'White',
  'money': 48.5,
  'dob': '1971-10-23',
  'employee_source': 'Glassdoor',
  'id': 905013738,
  'state': 'MA',
  'employment_status': 'Terminated for Cause',
 

In [500]:
s = qa.build_search(index='user_data_3')
len(s.query(sex='Male', size=300).execute())

10

In [496]:
users

[{'rft': 'no-call, no-show',
  'performance_score': 'N/A- too early to review',
  'citizendesc': 'US Citizen',
  'manager': 'Simon Roup',
  'sex': 'Male',
  'maritaldesc': 'Married',
  'emp_name': 'Hernandez, Daniff',
  'dot': '2015-02-22',
  'last_name': 'Hernandez',
  'racedesc': 'Black or African American',
  'money': 40.1,
  'dob': '1986-08-07',
  'employee_source': 'Employee Referral',
  'id': 1410071156,
  'state': 'MA',
  'employment_status': 'Terminated for Cause',
  'position': 'Database Administrator',
  'doh': '2015-02-16',
  'department': 'IT/IS',
  'first_name': 'Daniff',
  'age': 31},
 {'rft': 'N/A - still employed',
  'performance_score': 'N/A- too early to review',
  'citizendesc': 'US Citizen',
  'manager': 'Simon Roup',
  'sex': 'Male',
  'maritaldesc': 'Married',
  'emp_name': 'Rogers, Ivan',
  'dot': '-1',
  'last_name': 'Rogers',
  'racedesc': 'White',
  'money': 42.2,
  'dob': '1986-08-26',
  'employee_source': 'Pay Per Click - Google',
  'id': 1203032255,
  'stat

In [193]:
qa_out = qa.get(index='user_data', sex='Male')
qa_out

[{'rft': 'no-call, no-show',
  'performance_score': 'N/A- too early to review',
  'citizendesc': 'US Citizen',
  'sex': 'Male',
  'maritaldesc': 'Married',
  'emp_name': 'Hernandez, Daniff',
  'dot': '2/22/15',
  'man_name': 'Simon Roup',
  'racedesc': 'Black or African American',
  'money': 40.1,
  'dob': '8/7/86',
  'employee_source': 'Employee Referral',
  'id': 1410071156,
  'state': 'MA',
  'employment_status': 'Terminated for Cause',
  'position': 'Database Administrator',
  'doh': '2/16/15',
  'department': 'IT/IS',
  'age': 31},
 {'rft': 'N/A - still employed',
  'performance_score': 'N/A- too early to review',
  'citizendesc': 'US Citizen',
  'sex': 'Male',
  'maritaldesc': 'Married',
  'emp_name': 'Rogers, Ivan',
  'dot': -1,
  'man_name': 'Simon Roup',
  'racedesc': 'White',
  'money': 42.2,
  'dob': '8/26/86',
  'employee_source': 'Pay Per Click - Google',
  'id': 1203032255,
  'state': 'MA',
  'employment_status': 'Active',
  'position': 'Database Administrator',
  'doh': 

In [203]:
s = qa.build_search(index='user_data_3')

In [532]:
len(s.filter(field="age", lt='50').execute(size=300))

264

In [199]:
np.sum([emp['money'] for emp in qa_out])

367.01

In [413]:
get_now_t_str()

'2019-06-05'

In [408]:
d_to_ymd(get_ago_t(years=5))

'2014-06-11'

In [425]:
users = qa.get(index='user_data_3',size=301)
users

[{'rft': 'N/A - still employed',
  'performance_score': 'N/A- too early to review',
  'citizendesc': 'US Citizen',
  'manager': 'Brandon R. LeBlanc',
  'sex': 'Female',
  'maritaldesc': 'Single',
  'emp_name': 'Singh, Nan ',
  'dot': '-1',
  'last_name': 'Singh',
  'racedesc': 'White',
  'money': 16.56,
  'dob': '1988-05-19',
  'employee_source': 'Website Banner Ads',
  'id': 1307059817,
  'state': 'MA',
  'employment_status': 'Active',
  'position': 'Administrative Assistant',
  'doh': '2015-05-01',
  'department': 'Admin Offices',
  'first_name': 'Nan',
  'age': 29},
 {'rft': 'N/A - still employed',
  'performance_score': '90-day meets',
  'citizendesc': 'US Citizen',
  'manager': 'Simon Roup',
  'sex': 'Male',
  'maritaldesc': 'Married',
  'emp_name': 'Simard, Kramer',
  'dot': '-1',
  'last_name': 'Simard',
  'racedesc': 'White',
  'money': 30.2,
  'dob': '1970-02-08',
  'employee_source': 'Employee Referral',
  'id': 808010278,
  'state': 'MA',
  'employment_status': 'Active',
  '

In [226]:
filter_qa_out('sex','Male', users)

[{'rft': 'N/A - still employed',
  'performance_score': '90-day meets',
  'citizendesc': 'US Citizen',
  'manager': 'Simon Roup',
  'sex': 'Male',
  'maritaldesc': 'Married',
  'emp_name': 'Simard, Kramer',
  'dot': '-1',
  'last_name': 'Simard',
  'racedesc': 'White',
  'money': 30.2,
  'dob': '2/8/70',
  'employee_source': 'Employee Referral',
  'id': 808010278,
  'state': 'MA',
  'employment_status': 'Active',
  'position': 'Database Administrator',
  'doh': '1/5/15',
  'department': 'IT/IS',
  'first_name': 'Kramer',
  'age': 47},
 {'rft': 'N/A - still employed',
  'performance_score': 'Fully Meets',
  'citizendesc': 'US Citizen',
  'manager': 'Eric Dougall',
  'sex': 'Male',
  'maritaldesc': 'Single',
  'emp_name': 'Clayton, Rick',
  'dot': '-1',
  'last_name': 'Clayton',
  'racedesc': 'White',
  'money': 28.99,
  'dob': '9/5/85',
  'employee_source': 'Glassdoor',
  'id': 1301052902,
  'state': 'MA',
  'employment_status': 'Active',
  'position': 'IT Support',
  'doh': '9/5/12',
 

In [ ]:
Also, entering the size parameter on the execute function means that we will not be able to add on another filter after the execute. 